<a href="https://colab.research.google.com/github/architb1703/Toxic_Span/blob/Abhay/Improved_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install jsonlines 

In [1]:
import pandas as pd
import numpy as np
import re
# import jsonlines
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data
import json

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
cd /content/gdrive/'My Drive'/'Toxic Spam'

/content/gdrive/My Drive/Toxic Spam


In [ ]:
# Libraries

import matplotlib.pyplot as plt
import pandas as pd
import torch

# Preliminaries

from torchtext.data import Field, TabularDataset, BucketIterator

# Models

import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

# Training

import torch.optim as optim

# Evaluation

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,f1_score
import seaborn as sns

In [ ]:
data.head()

In [ ]:
data_train=pd.read_pickle('./processed/finaltrain.pkl')
data_val=pd.read_pickle('./processed/finaldev.pkl')
print(len(data_train),len(data_val))

6351 794


In [ ]:
###IGNORE THIS CELLLLL
# y=len(data)
# data=data.sample(frac=1)
# train_data= data[:int(8*y/10)]
# val_data=data[int(8*y/10):int(9*y/10)]
# test_data=data[int(9*y/10):]
# print(len(train_data),len(val_data),len(test_data))

6351 794 794


In [ ]:
X_train=data_train['token_final'].to_list()
Y_train=data_train['target_final'].to_list()
X_val=data_val['token_final'].tolist()
Y_val=data_val['target_final'].tolist()
print(X_train[0],Y_train[0])
print(X_val[0],Y_val[0])


train_data = []
for i in range(len(X_train)):
  train_data.append({"text" : X_train[i], "label" : Y_train[i]})

with jsonlines.open('./processed/train.json', mode='w') as f:
  f.write_all(train_data)

val_data = []
for i in range(len(X_val)):
  val_data.append({"text" : X_val[i], "label" : Y_val[i]})

with jsonlines.open('./processed/val.json', mode='w') as f:
  f.write_all(val_data)


['only', 'a', 'fool', 'would', 'not', 'be', 'against', 'this', 'president', 'he', 'is', 'massively', 'and', 'obviously', 'unfit', 'for', 'office'] [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['fucking', 'leftist', 'hebes', 'always', 'finding', 'the', 'dirty', 'way', 'to', 'find', 'dirt', 'and', 'make', 'it', 'salacious'] [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
class Corpus(object):
  def __init__(self, input_folder, min_word_freq, batch_size,device):
    # list all the fields
    self.word_field = Field(lower=True)
    self.label_field = Field(unk_token=None,pad_token=None)
    # create dataset using built-in parser from torchtext
    self.train_data, self.val_data = TabularDataset.splits(
        path=input_folder,
        train="train.json",
        validation="val.json",
        format="json",
        fields={'text':("text", self.word_field),'label':("label",self.label_field)}
    )
    # convert fields to vocabulary list
    # 
    self.word_field.build_vocab(self.train_data.text, vectors='glove.twitter.27B.200d', unk_init=torch.Tensor.normal_,min_freq=min_word_freq)
    self.label_field.build_vocab(self.train_data.label)
    # create iterator for batch input
    self.train_iter, self.val_iter = BucketIterator.splits(
        datasets=(self.train_data, self.val_data),
        batch_size=batch_size,
        sort_within_batch = True,
        sort_key = lambda x: len(x.text),
        device=device
    )
    # prepare padding index to be ignored during model training/evaluation
    self.word_pad_idx = self.word_field.vocab.stoi[self.word_field.pad_token]
    # self.label_pad_idx = self.tag_field.vocab.stoi[self.label_field.pad_token]

In [ ]:
device2 = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
corpus = Corpus(
    input_folder="./processed/",
    min_word_freq=2,  
    batch_size=64,
    device = device2
)
print(f"Train set: {len(corpus.train_data)} sentences")
print(f"Val set: {len(corpus.val_data)} sentences")

Train set: 6351 sentences
Val set: 794 sentences


In [ ]:
print(len(corpus.word_field.vocab))

8815


In [ ]:
class BiLSTM(nn.Module):

  def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, lstm_layers,
               emb_dropout, lstm_dropout, fc_dropout, word_pad_idx):
    super().__init__()
    self.embedding_dim = embedding_dim
    # LAYER 1: Embedding
    self.embedding = nn.Embedding(
        num_embeddings=input_dim, 
        embedding_dim=embedding_dim, 
        padding_idx=word_pad_idx
    )
    self.emb_dropout = nn.Dropout(emb_dropout)
    # LAYER 2: BiLSTM
    self.lstm = nn.LSTM(
        input_size=embedding_dim,
        hidden_size=hidden_dim,
        num_layers=lstm_layers,
        bidirectional=True,
        dropout=lstm_dropout if lstm_layers > 1 else 0
    )
    # LAYER 3: Fully-connected
    self.fc_dropout = nn.Dropout(fc_dropout)
    self.fc = nn.Linear(hidden_dim * 2, output_dim)  # times 2 for bidirectional


  def forward(self, sentence):
    # sentence = [sentence length, batch size]
    # embedding_out = [sentence length, batch size, embedding dim]
    embedding_out = self.emb_dropout(self.embedding(sentence))
    # lstm_out = [sentence length, batch size, hidden dim * 2]
    lstm_out, _ = self.lstm(embedding_out)
    # toxic_out = [sentence length, batch size, output dim]
    toxic_out = self.fc(self.fc_dropout(lstm_out))

    toxic_out2=torch.sigmoid(toxic_out)
    return toxic_out2

  def init_weights(self):
    # to initialize all parameters from normal distribution
    # helps with converging during training
    for name, param in self.named_parameters():
      nn.init.normal_(param.data, mean=0, std=0.1)

  def init_embeddings(self, word_pad_idx,pretrained=None):
    # initialize embedding for padding as zero
    self.embedding.weight.data[word_pad_idx] = torch.zeros(self.embedding_dim)
    if pretrained is not None:
      self.embedding = nn.Embedding.from_pretrained(
                embeddings=torch.as_tensor(pretrained),
                padding_idx=word_pad_idx,
                freeze=True
      )

  def count_parameters(self):
    return sum(p.numel() for p in self.parameters() if p.requires_grad)

In [ ]:
bilstm = BiLSTM(
    input_dim=len(corpus.word_field.vocab),
    embedding_dim=200,
    hidden_dim=64,
    output_dim=1,
    lstm_layers=2,
    emb_dropout=0,
    lstm_dropout=0,
    fc_dropout=0, 
    word_pad_idx=corpus.word_pad_idx
)
bilstm.init_weights()
# bilstm.init_embeddings(word_pad_idx=corpus.word_pad_idx)
bilstm.init_embeddings(
    word_pad_idx=corpus.word_pad_idx,
    pretrained=corpus.word_field.vocab.vectors
)
print(f"The model has {bilstm.count_parameters():,} trainable parameters.")
print(bilstm)

The model has 235,649 trainable parameters.
BiLSTM(
  (embedding): Embedding(8815, 200, padding_idx=1)
  (emb_dropout): Dropout(p=0, inplace=False)
  (lstm): LSTM(200, 64, num_layers=2, bidirectional=True)
  (fc_dropout): Dropout(p=0, inplace=False)
  (fc): Linear(in_features=128, out_features=1, bias=True)
)


In [ ]:
print(len(corpus.word_field.vocab))

6203


# TRAINER 

In [ ]:
import time

In [ ]:
class Toxicspan(object):

  def __init__(self, model, data, optimizer_cls, loss_fn_cls):
    self.model = model
    self.data = data
    self.optimizer = optimizer_cls(model.parameters())
    self.loss_fn = loss_fn_cls()

  def accuracy(self, preds, y):
    # rounded_preds=preds
    rounded_preds = torch.round(preds)
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return(acc)
  def sigmoid(self,x):
    return 1/(1 + np.exp(-np.array(x)))
  def f1score(self,preds,y):
    # rounded_preds = torch.round(preds)
    return f1_score(y,preds,average='binary')    

  def epoch(self):
      epoch_loss = 0
      epoch_acc = 0true_label
      self.model.train()
      batch_f1=[]
      true_labels=[]
      pred_labels=[]
      for batch in self.data.train_iter:
        text = batch.text
        true_label = batch.label
        self.optimizer.zero_grad()
        pred_label = self.model(text)
        # to calculate the loss and accuracy, we flatten both prediction and true tags
        # flatten pred_tags to [sent len, batch size, output dim]
        pred_label = pred_label.view(-1)
        # flatten true_tags to [sent len * batch size]
        true_label = true_label.view(-1)
        # print(true_label.shape,pred_label.shape)
        batch_loss = self.loss_fn(pred_label, true_label.type(torch.FloatTensor))
        batch_acc = self.accuracy(pred_label, true_label)
        # batch_f1.append(f1_score(true_label,torch.round(pred_label).detach().numpy(),average='binary'))
        true_labels.extend(true_label)
        with torch.no_grad():
          pred_labels.extend((torch.round(pred_label)))
        batch_loss.backward()
        self.optimizer.step()
        epoch_loss += batch_loss.item()
        epoch_acc += batch_acc.item()
      # print(pred_labels[0],true_labels[0])
      # print(true_label,torch.round(pred_label))
      # print(type(pred_labels[0]),type(true_labels[0]))
      # print(true_labels[:10],pred_labels[0:10])
      # batch_f1=0
      batch_f1=self.f1score(pred_labels,true_labels)
      return epoch_loss / len(self.data.train_iter), epoch_acc / len(self.data.train_iter), batch_f1

  def evaluate(self, iterator):
      epoch_loss = 0
      epoch_acc = 0
      true_labels=[]
      pred_labels=[]
      self.model.eval()
      with torch.no_grad():
          # similar to epoch() but model is in evaluation mode and no backprop
          for batch in iterator:
              text = batch.text
              true_label = batch.label
              pred_label = self.model(text)
              pred_label = pred_label.view(-1)
              true_label = true_label.view(-1)
              batch_loss = self.loss_fn(pred_label, true_label.type(torch.FloatTensor))
              batch_acc = self.accuracy(pred_label, true_label)
              epoch_loss += batch_loss.item()
              epoch_acc += batch_acc.item()
              pred_labels.extend(torch.round(pred_label))
              true_labels.extend(true_label)
          # print(true_labels[:10],pred_labels[0:10])
          batch_f1=self.f1score(pred_labels,true_labels)
          print(confusion_matrix(true_labels,pred_labels))
      return epoch_loss / len(iterator), epoch_acc / len(iterator), batch_f1

  # main training sequence
  def train(self, n_epochs):
    for epoch in range(n_epochs):
        start=time.time()
        train_loss, train_acc, train_f1 = self.epoch()
        final=time.time()
        print(f"\tTrn Loss: {train_loss:.3f} | Trn Acc: {train_acc * 100:.2f}% | Trn F1score: {train_f1:.5f}")
        print("Epoch time:",final-start)
        val_loss, val_acc,val_f1 = self.evaluate(self.data.val_iter)
        print(f"\tVal Loss: {val_loss:.3f} | Val Acc: {val_acc * 100:.2f}% | Val F1 :{val_f1:.5f}")
    # test_loss, test_acc = self.evaluate(self.data.test_iter)
    # print(f"Test Loss: {test_loss:.3f} |  Test Acc: {test_acc * 100:.2f}%")
  

In [ ]:
# this will continue training if the model has been trained before.
# to restart training, run the bilstm creation cell (2 cells above) once again.
import torch.optim as optim
toxic = Toxicspan(
  model=bilstm,
  data=corpus,
  optimizer_cls=optim.Adam,
  loss_fn_cls=nn.BCEWithLogitsLoss
)
toxic.train(10)

	Trn Loss: 0.707 | Trn Acc: 85.90% | Trn F1score: 0.02074
Epoch time: 19.301296949386597
[[31685     0]
 [ 2491     0]]
	Val Loss: 0.693 | Val Acc: 88.49% | Val F1 :0.00000
	Trn Loss: 0.693 | Trn Acc: 86.92% | Trn F1score: 0.00000
Epoch time: 18.884336948394775
[[31685     0]
 [ 2491     0]]
	Val Loss: 0.693 | Val Acc: 88.49% | Val F1 :0.00000
	Trn Loss: 0.693 | Trn Acc: 86.92% | Trn F1score: 0.00000
Epoch time: 18.872860193252563
[[31685     0]
 [ 2491     0]]
	Val Loss: 0.693 | Val Acc: 88.49% | Val F1 :0.00000
	Trn Loss: 0.692 | Trn Acc: 87.40% | Trn F1score: 0.00800
Epoch time: 18.71992588043213
[[31679     6]
 [ 2486     5]]
	Val Loss: 0.692 | Val Acc: 88.47% | Val F1 :0.00400
	Trn Loss: 0.684 | Trn Acc: 89.92% | Trn F1score: 0.24744
Epoch time: 18.63112783432007
[[31563   122]
 [ 1974   517]]
	Val Loss: 0.684 | Val Acc: 91.24% | Val F1 :0.33035
	Trn Loss: 0.679 | Trn Acc: 91.22% | Trn F1score: 0.38008
Epoch time: 19.05133318901062
[[31541   144]
 [ 1860   631]]
	Val Loss: 0.682 |